# Zenith Online - Data Transformation Pipeline (Unity Catalog)

This notebook implements the full Bronze-Silver-Gold ETL pipeline for Zenith Online, fully integrated with Databricks Unity Catalog. It is designed to be run after the data generator has populated the landing zone volume.

## Pipeline Initialization

Import required libraries and set up Spark session. Define all paths and table names for Unity Catalog integration.

In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
from pyspark.sql.window import Window
import pandas as pd

# In Databricks, the SparkSession `spark` is already created for you.

In [0]:
# Define the three-level namespace for Unity Catalog
CATALOG_NAME = "zenith_online"

SYSTEM_SCHEMA = "_system"
LANDING_SCHEMA = "00_landing"
BRONZE_SCHEMA = "01_bronze"
SILVER_SCHEMA = "02_silver"
GOLD_SCHEMA = "03_gold"

# The data generator wrote data to these UC Volume paths
RAW_STREAMING_PATH = f"/Volumes/{CATALOG_NAME}/{LANDING_SCHEMA}/streaming/user_events"
RAW_BATCH_CUSTOMERS_PATH = f"/Volumes/{CATALOG_NAME}/{LANDING_SCHEMA}/batch/customers"
RAW_BATCH_PRODUCTS_PATH = f"/Volumes/{CATALOG_NAME}/{LANDING_SCHEMA}/batch/products"

# Define UC Volume paths for streaming checkpoints and schema metadata
CHECKPOINT_BASE_PATH = f"/Volumes/{CATALOG_NAME}/{SYSTEM_SCHEMA}/checkpoints"
SCHEMA_BASE_PATH = f"/Volumes/{CATALOG_NAME}/{SYSTEM_SCHEMA}/schemas"

# Full table names
BRONZE_EVENTS_TABLE = f"{CATALOG_NAME}.{BRONZE_SCHEMA}.bronze_user_events"
BRONZE_CUSTOMERS_TABLE = f"{CATALOG_NAME}.{BRONZE_SCHEMA}.bronze_customer_profiles"
BRONZE_PRODUCTS_TABLE = f"{CATALOG_NAME}.{BRONZE_SCHEMA}.bronze_product_details"
SILVER_TABLE = f"{CATALOG_NAME}.{SILVER_SCHEMA}.silver_sessionized_activity"
GOLD_DAILY_PRODUCT_TABLE = f"{CATALOG_NAME}.{GOLD_SCHEMA}.gold_daily_product_performance"
GOLD_CUSTOMER_SUMMARY_TABLE = f"{CATALOG_NAME}.{GOLD_SCHEMA}.customer_purchase_summary"

## Unity Catalog Environment Setup

Create the catalog and schemas if they do not exist. This ensures all downstream tables are created in the correct namespace.

In [0]:
%run "/Workspace/Users/s@jlasak.com/zenith/Setup Environment"

## Populate initial data

In [0]:
from data_generator import generate_batch_data, generate_streaming_events
generate_batch_data()

generate_streaming_events(infinite=False, stream_events_per_batch=50000)
# Infinite streaming is disabled by infinite=False, which means it will just generate one batch of data and stop.

[2025-08-02 09:54:52.492210] Starting generation of initial batch data...
[2025-08-02 09:54:52.809972] Successfully wrote 3000 customers to: /Volumes/zenith_online/00_landing/batch/customers/customer_profiles.csv
[2025-08-02 09:54:52.940546] Successfully wrote 1500 products to: /Volumes/zenith_online/00_landing/batch/products/product_details.parquet
--------------------------------------------------
[2025-08-02 09:54:53.057842] Starting event stream generation...
Data will be written to: /Volumes/zenith_online/00_landing/streaming/user_events
Data Skew active: 75.0% of 'view_product' events target [101, 210, 315, 420, 55]
Streaming mode: single batch
--------------------------------------------------
[2025-08-02 09:54:54.939629] Wrote batch of 50000 events to /Volumes/zenith_online/00_landing/streaming/user_events/events_20250802_095453_703194.json


## 01 - Bronze Layer: Raw Data Ingestion

### Streaming User Events
Ingest raw user event data from JSON files using Auto Loader and write to a Delta table in the Bronze schema.

In [0]:
print("Starting Bronze Layer: Streaming Events Ingestion")

event_schema = StructType([
    StructField("event_id", StringType(), True),
    StructField("event_timestamp", StringType(), True),
    StructField("user_id", IntegerType(), True),
    StructField("event_type", StringType(), True),
    StructField("product_id", IntegerType(), True),
    StructField("session_id", StringType(), True)
])

bronze_events_df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", f"{SCHEMA_BASE_PATH}/bronze_events")
    .schema(event_schema)
    .load(RAW_STREAMING_PATH)
    .withColumn("ingestion_timestamp", F.current_timestamp())
    .withColumn("event_dt", F.to_timestamp("event_timestamp"))
)

(
    bronze_events_df.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", f"{CHECKPOINT_BASE_PATH}/bronze_events")
    .trigger(availableNow=True) 
    .toTable(BRONZE_EVENTS_TABLE)
)

print(f"Bronze Layer: Streaming events processing complete. Table `{BRONZE_EVENTS_TABLE}` is updated.")

Starting Bronze Layer: Streaming Events Ingestion
Bronze Layer: Streaming events processing complete. Table `zenith_online.01_bronze.bronze_user_events` is updated.


### Batch Customer & Product Data
Ingest customer profiles (CSV) and product details (Parquet) into Bronze tables for further enrichment.

In [0]:
print("Starting Bronze Layer: Batch Ingestion")

customer_schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("signup_date", StringType(), True),
    StructField("location", StringType(), True)
])
customers_df = spark.read.format("csv").schema(customer_schema).option("header", "true").load(RAW_BATCH_CUSTOMERS_PATH)
customers_df.write.format("delta").mode("overwrite").saveAsTable(BRONZE_CUSTOMERS_TABLE)
print(f"Bronze Layer: Wrote customer profiles to `{BRONZE_CUSTOMERS_TABLE}`.")

products_df = spark.read.format("parquet").load(RAW_BATCH_PRODUCTS_PATH)
products_df.write.format("delta").mode("overwrite").saveAsTable(BRONZE_PRODUCTS_TABLE)
print(f"Bronze Layer: Wrote product details to `{BRONZE_PRODUCTS_TABLE}`.")

Starting Bronze Layer: Batch Ingestion
Bronze Layer: Wrote customer profiles to `zenith_online.01_bronze.bronze_customer_profiles`.
Bronze Layer: Wrote product details to `zenith_online.01_bronze.bronze_product_details`.


## 02 - Silver Layer - Clean, Enrich, and De-duplicate

**Goal:** Read from the Bronze UC tables, enrich the data, and write the cleaned result to a Silver UC table (`zenith_online.02_silver.silver_sessionized_activity`).

### Define Region Categorization Logic

Create a Pandas UDF to categorize customer locations into regions for downstream analytics.

In [0]:
@F.pandas_udf(StringType())
def categorize_region(locations: pd.Series) -> pd.Series:
    east_coast = ['New York', 'Philadelphia']
    west_coast = ['Los Angeles', 'San Diego']
    def get_region(location):
        if location in east_coast: return 'East Coast'
        elif location in west_coast: return 'West Coast'
        else: return 'Central'
    return locations.apply(get_region)

print("Silver Layer: Pandas UDF `categorize_region` created.")


Silver Layer: Pandas UDF `categorize_region` created.


### Silver Layer Transformation

Deduplicate streaming events, join with customer and product data, enrich with region, and write to the Silver Delta table.

In [0]:
print("Starting Silver Layer: Enriching and Cleaning Events")

bronze_events_stream_df = spark.readStream.table(BRONZE_EVENTS_TABLE)
customers_df = spark.read.table(BRONZE_CUSTOMERS_TABLE)
products_df = spark.read.table(BRONZE_PRODUCTS_TABLE)

deduplicated_stream_df = (
    bronze_events_stream_df
        .withWatermark("event_dt", "3 minutes")
        .dropDuplicates(["event_id"])
)

enriched_df = (
    deduplicated_stream_df
    .join(
        F.broadcast(products_df), 
        deduplicated_stream_df.product_id == products_df.product_id,
        "inner"
    )
    .join(
        customers_df,
        deduplicated_stream_df.user_id == customers_df.customer_id,
        "left"
    )
    .withColumn("region", categorize_region(F.col("location")))
    .withColumnRenamed("event_type", "action")
    .withColumn("event_date", F.to_date(F.col("event_dt")))
    .select(
        "event_id", "event_dt", "event_date", "action",
        deduplicated_stream_df.user_id, "region",
        deduplicated_stream_df.product_id, "product_name", "category", "price"
    )
)

(
    enriched_df.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", f"{CHECKPOINT_BASE_PATH}/silver_activity")
    .trigger(availableNow=True)
    .toTable(SILVER_TABLE)
)

print(f"Silver Layer: Enriched activity stream processing complete. Table `{SILVER_TABLE}` is updated.")

Starting Silver Layer: Enriching and Cleaning Events
Silver Layer: Enriched activity stream processing complete. Table `zenith_online.02_silver.silver_sessionized_activity` is updated.


## 03 - Gold Layer: Business-Focused Aggregations

Produce analytics-ready tables for business intelligence and reporting.

### Daily Product Performance (Handling Data Skew)

Aggregate Silver data to compute daily product metrics, using salting to mitigate data skew for popular products.

In [0]:
print("Starting Gold Layer: Daily Product Performance Aggregation")
silver_df = spark.read.table(SILVER_TABLE)

SALTING_FACTOR = 5
salted_df = silver_df.withColumn("salt", (F.rand() * SALTING_FACTOR).cast("int"))

salted_agg_df = (
    salted_df.groupBy("event_date", "product_id", "product_name", "category", "salt")
    .agg(
        F.count(F.when(F.col("action") == "view_product", 1)).alias("views"),
        F.count(F.when(F.col("action") == "add_to_cart", 1)).alias("adds_to_cart"),
        F.count(F.when(F.col("action") == "purchase", 1)).alias("purchases"),
        F.sum(F.when(F.col("action") == "purchase", F.col("price")).otherwise(0)).alias("daily_revenue")
    )
)

daily_product_performance_df = (
    salted_agg_df.groupBy("event_date", "product_id", "product_name", "category")
    .agg(
        F.sum("views").alias("total_views"),
        F.sum("adds_to_cart").alias("total_adds_to_cart"),
        F.sum("purchases").alias("total_purchases"),
        F.sum("daily_revenue").alias("total_revenue")
    )
)

window_spec = Window.partitionBy("event_date", "category").orderBy(F.col("total_revenue").desc())
final_product_gold_df = daily_product_performance_df.withColumn("revenue_rank", F.rank().over(window_spec))

(
    final_product_gold_df.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .partitionBy("event_date")
    .saveAsTable(GOLD_DAILY_PRODUCT_TABLE)
)

print(f"Gold Layer: Wrote to `{GOLD_DAILY_PRODUCT_TABLE}` after handling skew.")


Starting Gold Layer: Daily Product Performance Aggregation
Gold Layer: Wrote to `zenith_online.03_gold.gold_daily_product_performance` after handling skew.


In [0]:
display(final_product_gold_df)

event_date,product_id,product_name,category,total_views,total_adds_to_cart,total_purchases,total_revenue,revenue_rank
2025-08-01,409,Modern Desk,Electronics,65,63,39,33847.31999999999,1
2025-08-01,403,Classic Table,Electronics,43,57,46,32296.60000000001,2
2025-08-01,339,Ergonomic Lamp,Electronics,44,52,38,32196.26000000001,3
2025-08-01,148,Durable Table,Electronics,47,54,40,31298.000000000007,4
2025-08-01,241,Sleek Keyboard,Electronics,59,70,36,31001.76,5
2025-08-01,23,Durable Monitor,Electronics,45,69,35,30595.6,6
2025-08-01,482,Lightweight Keyboard,Electronics,34,53,34,30391.919999999995,7
2025-08-01,300,Heavy Duty Chair,Electronics,53,66,34,30015.879999999997,8
2025-08-01,488,Rustic Keyboard,Electronics,64,67,33,29267.03999999999,9
2025-08-01,362,Heavy Duty Lamp,Electronics,43,71,38,29204.52000000001,10


### Customer Purchase Summary

Aggregate customer-level purchase metrics and perform a data quality check for missing regions.

In [0]:
print("Starting Gold Layer: Customer Purchase Summary")
silver_df = spark.read.table(SILVER_TABLE)

# --- Data Quality Check (Serverless-Compatible Method) ---
# # Accumulators are not available on serverless compute as they require direct sparkContext access.
# unknown_location_count = spark.sparkContext.accumulator(0)
# def count_unknown_locations(region):
#     if region is None:
#         global unknown_location_count
#         unknown_location_count += 1
#     return region
# count_unknown_udf = F.udf(count_unknown_locations, StringType())
# The best practice is to perform a direct count() action on a filtered DataFrame.
# This achieves the same goal of gathering a metric from the data.
unknown_location_count = silver_df.filter(F.col("region").isNull()).count()

print(f"Data Quality Check: Found {unknown_location_count} events with unknown customer locations.")

customer_summary_df = (
    silver_df.filter(F.col("action") == 'purchase')
    .groupBy("user_id", "region")
    .agg(
        F.sum("price").alias("total_purchase_value"),
        F.count("event_id").alias("total_purchases"),
        F.approx_count_distinct("product_id").alias("distinct_products_purchased"),
        F.max("event_dt").alias("last_purchase_timestamp")
    )
    .orderBy(F.col("total_purchase_value").desc())
)

(
    customer_summary_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(GOLD_CUSTOMER_SUMMARY_TABLE)
)
print(f"Gold Layer: Wrote to `{GOLD_CUSTOMER_SUMMARY_TABLE}`.")

Starting Gold Layer: Customer Purchase Summary
Data Quality Check: Found 0 events with unknown customer locations.
Gold Layer: Wrote to `zenith_online.03_gold.customer_purchase_summary`.


## Answering Business Questions

Now that our Gold tables exist in Unity Catalog, we can query them directly with SQL.


### Business Question 1: 
What are the top 5 most purchased products for the most recent day?

Query the Gold product performance table to find the most popular products.

In [0]:
print("--- Top 5 Purchased Products (Most Recent Day) ---")
spark.sql(f"""
    SELECT 
        event_date,
        product_name,
        category,
        total_purchases,
        total_revenue
    FROM {GOLD_DAILY_PRODUCT_TABLE}
    --WHERE event_date = (SELECT MAX(event_date) FROM {GOLD_DAILY_PRODUCT_TABLE})
    ORDER BY total_purchases DESC
    LIMIT 5
""").show()


--- Top 5 Purchased Products (Most Recent Day) ---
+----------+---------------+-----------+---------------+------------------+
|event_date|   product_name|   category|total_purchases|     total_revenue|
+----------+---------------+-----------+---------------+------------------+
|2025-08-01|Ergonomic Clock|  Furniture|             47|          16591.47|
|2025-08-01|  Classic Table|Electronics|             46| 32296.60000000001|
|2025-08-01|  Sleek Monitor| Home Decor|             45|24537.150000000012|
|2025-08-01|  Durable Clock|  Furniture|             44|30912.639999999992|
|2025-08-01|   Modern Chair|Electronics|             43|          16798.81|
+----------+---------------+-----------+---------------+------------------+



### Business Question 2: 
What is the daily sales revenue per category?

Summarize daily revenue by product category for trend analysis.

In [0]:
print("--- Daily Sales Revenue per Category ---")
spark.sql(f"""
    SELECT 
        event_date,
        category,
        SUM(total_revenue) as category_revenue
    FROM {GOLD_DAILY_PRODUCT_TABLE}
    GROUP BY event_date, category
    ORDER BY event_date DESC, category_revenue DESC
""").show(truncate=False)


--- Daily Sales Revenue per Category ---
+----------+---------------+------------------+
|event_date|category       |category_revenue  |
+----------+---------------+------------------+
|2025-08-01|Electronics    |2427121.380000002 |
|2025-08-01|Home Decor     |2259460.6800000016|
|2025-08-01|Office Supplies|2208186.530000003 |
|2025-08-01|Furniture      |2205430.8200000003|
+----------+---------------+------------------+



### Business Question 3:
What are the top 10 Customers by purchase value?

Identify the highest-value customers based on total purchase value.

In [0]:
print("--- Top 10 Customers by Purchase Value ---")
spark.sql(f"""
    SELECT
        user_id,
        region,
        total_purchase_value,
        total_purchases,
        last_purchase_timestamp
    FROM {GOLD_CUSTOMER_SUMMARY_TABLE}
    ORDER BY total_purchase_value DESC
    LIMIT 10
""").show()

--- Top 10 Customers by Purchase Value ---
+-------+----------+--------------------+---------------+-----------------------+
|user_id|    region|total_purchase_value|total_purchases|last_purchase_timestamp|
+-------+----------+--------------------+---------------+-----------------------+
|    255|East Coast|             16920.4|             35|   2025-08-01 19:23:...|
|    237|   Central|  16381.109999999997|             31|   2025-08-01 19:23:...|
|    596|   Central|  15461.760000000002|             28|   2025-08-01 19:22:...|
|    825|   Central|  15246.369999999997|             30|   2025-08-01 19:23:...|
|    207|West Coast|  14966.980000000001|             28|   2025-08-01 19:23:...|
|    232|   Central|            14936.57|             27|   2025-08-01 19:23:...|
|    419|   Central|            14830.93|             26|   2025-08-01 19:24:...|
|    234|West Coast|  14435.289999999999|             30|   2025-08-01 19:24:...|
|    803|   Central|             14402.0|             3

## End of Pipeline

The script has successfully processed data through the Bronze, Silver, and Gold layers within Unity Catalog, creating valuable business assets and demonstrating key capabilities tested in the Spark Developer certification exam.